# Modèle de Curtis et Hirschfelder

Le problème de Curtis and Hirschfelder s'écrit sous la forme :

\begin{equation*}
\left\{
\begin{aligned}
d_t y(t) & = k \, \big( -y(t) + \cos(t) \big) \quad \text{avec } k > 1\\
y(0)  & = y_0
\end{aligned}
\right.
\end{equation*}

In [ ]:
from mylib.model import curtiss_model
import mylib.integration as integration

import numpy as np

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row

from ipywidgets import interact, FloatSlider, IntSlider

from mylib.model import curtiss_model

from mylib.integration import forward_euler, backward_euler, rk2

import warnings
warnings.filterwarnings('ignore')

output_notebook(hide_banner=True)

## Solution exacte

La solution exacte s'écrit :

\begin{equation*}
y(t) = C \, e^{-k t} + \frac{k^2 \cos(t)}{1+k^2} + \frac{k \sin(t)}{1+k^2} \quad \text{avec } C=y_0-\frac{k^2}{1+k^2}  
\end{equation*}

In [ ]:
def show_exact_sol():

    yini = 2.
    tini = 0.
    tend = 1.5
    k = 50.
    
    cm = curtiss_model(k)
    fcn = cm.fcn
    
    texa = np.linspace(tini, tend, 500)
    yexa = cm.sol(yini, texa)
 
    fig_sol = figure(x_range=(tini, tend), y_range=(0.0, 2.), plot_height=400, plot_width=900,
                     title="Exact solution of Curtiss and Hirschfelder equation")
    plt_sol = fig_sol.line(texa, yexa, color="Crimson", line_width=2)
    
    show(fig_sol, notebook_handle=True)
    
    def update(k):
        cm = curtiss_model(k)
        fcn = cm.fcn
        yexa = cm.sol(yini, texa)
        plt_sol.data_source.data = dict(x=texa, y=yexa)
        push_notebook()

    interact(update, k=FloatSlider(min=1, max=200, value=k, step=5))
    
show_exact_sol() 

## Méthode d'Euler explicite

La méthode d'Euler explicite pour résoudre ${\mathrm d}_t y(t) = f(t,y)$ peut s'écrire :

\begin{equation*}
\left\{
\begin{aligned}
& y^0 = y_0 \\
& y^{n+1} = y^n + \Delta t \; f(t^n,y^n) \quad \text{where} \quad \Delta t = t^{n+1} - t^n
\end{aligned}
\right.
\end{equation*}

In [ ]:
def show_forward_euler_sol():

    yini = 2.
    tini = 0.
    tend = 1.5
    k = 50.
    
    cm = curtiss_model(k)
    fcn = cm.fcn
    
    texa = np.linspace(tini, tend, 500)
    yexa = cm.sol(yini, texa)
 
    nt = 50
    sol_num = integration.forward_euler(tini, tend, nt, yini, fcn)
    
    yerr = np.absolute(cm.sol(yini, sol_num.t) - sol_num.y[0])
    cm.sol(yini, sol_num.t)
    yy = cm.sol(yini, sol_num.t)
    
    fig_sol = figure(x_range=(tini, tend), y_range=(-0.25, 2.25), plot_height=300, plot_width=900, title="Solution")
    plt_sol = fig_sol.line(texa, yexa, color="Crimson", line_width=2)
    plt_num = fig_sol.x(sol_num.t, sol_num.y[0], line_width=2, size=8)
    plt_line_num = fig_sol.line(sol_num.t, sol_num.y[0], line_width=1, line_dash="dotted")
    fig_err = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Global error")
    plt_err = fig_err.x(sol_num.t, yerr, line_width=2, size=8)
    
    show(column(fig_sol,fig_err), notebook_handle=True)
    
    def update(nt, k):
        cm = curtiss_model(k)
        fcn = cm.fcn
        yexa = cm.sol(yini, texa)
        sol_num = integration.forward_euler(tini, tend, nt, yini, fcn)
        yerr = np.absolute(cm.sol(yini, sol_num.t) - sol_num.y[0])
        plt_sol.data_source.data = dict(x=texa, y=yexa)
        plt_num.data_source.data = dict(x=sol_num.t, y=sol_num.y[0])
        plt_line_num.data_source.data = dict(x=sol_num.t, y=sol_num.y[0])
        plt_err.data_source.data = dict(x=sol_num.t, y=yerr)
        push_notebook()

    interact(update, nt=IntSlider(min=10, max=200, value=nt, step=1, continuous_update=False),
             k=FloatSlider(min=5, max=200, value=k, step=5))
    
show_forward_euler_sol()

### Ordre de la méthode

In [ ]:
def show_forward_euler_order():

    yini = 2
    tini = 0.
    tend = 1.5
    cm = curtiss_model(k=50)
    fcn = cm.fcn

    nt = np.array([151, 1501, 15001, 150001])
    dt = (tend-tini)/(nt-1)
    
    err_l1   = np.zeros(nt.size)
    err_l2   = np.zeros(nt.size)
    err_linf = np.zeros(nt.size)

    for it, nti in enumerate(nt):

        sol_num = integration.forward_euler(tini, tend, nti, yini, fcn)
        yerr = np.absolute(cm.sol(yini, sol_num.t) - sol_num.y[0])
        
        err_l1[it]   = np.linalg.norm(yerr, 1) / (nti-1)
        err_l2[it]   = np.linalg.norm(yerr) / np.sqrt(nti-1)   
        err_linf[it] = np.linalg.norm(yerr, np.inf) 

    fig_err = figure(x_axis_type="log", y_axis_type="log", title="Error", plot_width=950, plot_height=400)
    fig_err.x(dt, err_l1, size=10, line_width=2, legend="l1 norm")
    fig_err.x(dt, err_l2, size=10, line_width=2, color="crimson", legend="l2 norm")
    fig_err.x(dt, err_linf, size=10, line_width=2, color="green", legend="linf norm")
    fig_err.line(dt, (err_l2[0]/dt[0])*dt, line_width=2, legend="slope 1", color="black", line_dash='dotted')

    fig_err.legend.location = "top_left"
    fig_err.legend.click_policy="hide"

    show(fig_err)
    
show_forward_euler_order()

## Méthode d'Euler implicite

La méthode d'Euler implicite pour résoudre ${\mathrm d}_t y(t) = f(t,y)$ peut s'écrire :

\begin{equation*}
\left\{
\begin{aligned}
& y^0 = y_0 \\
& y^{n+1} = y^n + \Delta t \; f(t^{n+1},y^{n+1}), \qquad \Delta t = t^{n+1} - t^n,
\end{aligned}
\right.
\end{equation*}

In [ ]:
def show_backward_euler_sol():

    yini = 2.
    tini = 0.
    tend = 1.5
    k = 50.
    
    cm = curtiss_model(k)
    fcn = cm.fcn
    
    texa = np.linspace(tini, tend, 500)
    yexa = cm.sol(yini, texa)
 
    nt = 50
    sol_num = integration.backward_euler(tini, tend, nt, yini, fcn)
    
    yerr = np.absolute(cm.sol(yini, sol_num.t) - sol_num.y[0])
    cm.sol(yini, sol_num.t)
    yy = cm.sol(yini, sol_num.t)
    
    fig_sol = figure(x_range=(tini, tend), y_range=(-0.25, 2.25), plot_height=300, plot_width=900, title="Solution")
    plt_sol = fig_sol.line(texa, yexa, color="Crimson", line_width=2)
    plt_num = fig_sol.x(sol_num.t, sol_num.y[0], line_width=2, size=8)
    plt_line_num = fig_sol.line(sol_num.t, sol_num.y[0], line_width=1, line_dash="dotted")
    fig_err = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Global error")
    plt_err = fig_err.x(sol_num.t, yerr, line_width=2, size=8)
    
    show(column(fig_sol,fig_err), notebook_handle=True)
    
    def update(nt, k):
        cm = curtiss_model(k)
        fcn = cm.fcn
        yexa = cm.sol(yini, texa)
        sol_num = integration.backward_euler(tini, tend, nt, yini, fcn)
        yerr = np.absolute(cm.sol(yini, sol_num.t) - sol_num.y[0])
        plt_sol.data_source.data = dict(x=texa, y=yexa)
        plt_num.data_source.data = dict(x=sol_num.t, y=sol_num.y[0])
        plt_line_num.data_source.data = dict(x=sol_num.t, y=sol_num.y[0])
        plt_err.data_source.data = dict(x=sol_num.t, y=yerr)
        push_notebook()

    interact(update, nt=IntSlider(min=10, max=200, value=nt, step=1, continuous_update=False),
             k=FloatSlider(min=5, max=200, value=k, step=5))
    
show_backward_euler_sol()

### Ordre de la méthode

In [ ]:
def show_backward_euler_order():

    yini = 2
    tini = 0.
    tend = 1.5
    cm = curtiss_model(k=50)
    fcn = cm.fcn

    nt = np.array([151, 1501, 15001, 150001])
    dt = (tend-tini)/(nt-1)
    
    err_l1   = np.zeros(nt.size)
    err_l2   = np.zeros(nt.size)
    err_linf = np.zeros(nt.size)

    for it, nti in enumerate(nt):

        sol_num = integration.backward_euler(tini, tend, nti, yini, fcn)
        yerr = np.absolute(cm.sol(yini, sol_num.t) - sol_num.y[0])
        
        err_l1[it]   = np.linalg.norm(yerr, 1) / (nti-1)
        err_l2[it]   = np.linalg.norm(yerr) / np.sqrt(nti-1)   
        err_linf[it] = np.linalg.norm(yerr, np.inf) 

    fig_err = figure(x_axis_type="log", y_axis_type="log", title="Error", plot_width=950, plot_height=400)
    fig_err.x(dt, err_l1, size=10, line_width=2, legend="l1 norm")
    fig_err.x(dt, err_l2, size=10, line_width=2, color="crimson", legend="l2 norm")
    fig_err.x(dt, err_linf, size=10, line_width=2, color="green", legend="linf norm")
    fig_err.line(dt, (err_l2[0]/dt[0])*dt, line_width=2, legend="slope 1", color="black", line_dash='dotted')

    fig_err.legend.location = "top_left"
    fig_err.legend.click_policy="hide"

    show(fig_err)
            
show_backward_euler_order()

## Méthode de Runge Kutta d'ordre 2

La méthode de Runge Kutta d'ordre 2 pour résoudre ${\mathrm d}_t y(t) = f(t,y)$ peut s'écrire :

\begin{equation*}
\left\{
\begin{aligned}
& y^0 = y_0 \\
& y^{n+1} = y^n + \Delta t \; f \Big(t^n+\frac{\Delta t}{2},y^n + \frac{\Delta t}{2} f(t^n, y^n)\Big), \qquad \Delta t = t^{n+1} - t^n,
\end{aligned}
\right.
\end{equation*}

In [ ]:
def show_rk2_sol():

    yini = 2.
    tini = 0.
    tend = 1.5
    k = 50.
    
    cm = curtiss_model(k)
    fcn = cm.fcn
    
    texa = np.linspace(tini, tend, 500)
    yexa = cm.sol(yini, texa)
 
    nt = 50
    sol_num = integration.rk2(tini, tend, nt, yini, fcn)
    
    yerr = np.absolute(cm.sol(yini, sol_num.t) - sol_num.y[0])
    cm.sol(yini, sol_num.t)
    yy = cm.sol(yini, sol_num.t)
    
    fig_sol = figure(x_range=(tini, tend), y_range=(-0.25, 2.25), plot_height=300, plot_width=900, title="Solution")
    plt_sol = fig_sol.line(texa, yexa, color="Crimson", line_width=2)
    plt_num = fig_sol.x(sol_num.t, sol_num.y[0], line_width=2, size=8)
    plt_line_num = fig_sol.line(sol_num.t, sol_num.y[0], line_width=1, line_dash="dotted")
    fig_err = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Global error")
    plt_err = fig_err.x(sol_num.t, yerr, line_width=2, size=8)
    
    show(column(fig_sol,fig_err), notebook_handle=True)
    
    def update(nt, k):
        cm = curtiss_model(k)
        fcn = cm.fcn
        yexa = cm.sol(yini, texa)
        sol_num = integration.rk2(tini, tend, nt, yini, fcn)
        yerr = np.absolute(cm.sol(yini, sol_num.t) - sol_num.y[0])
        plt_sol.data_source.data = dict(x=texa, y=yexa)
        plt_num.data_source.data = dict(x=sol_num.t, y=sol_num.y[0])
        plt_line_num.data_source.data = dict(x=sol_num.t, y=sol_num.y[0])
        plt_err.data_source.data = dict(x=sol_num.t, y=yerr)
        push_notebook()

    interact(update, nt=IntSlider(min=10, max=200, value=nt, step=1, continuous_update=False),
             k=FloatSlider(min=5, max=200, value=k, step=5))
    
show_rk2_sol()

### Ordre de la méthode

In [ ]:
def show_rk2_order():

    yini = 2
    tini = 0.
    tend = 1.5
    cm = curtiss_model(k=50)
    fcn = cm.fcn

    nt = np.array([151, 1501, 15001, 150001])
    dt = (tend-tini)/(nt-1)
    
    err_l1   = np.zeros(nt.size)
    err_l2   = np.zeros(nt.size)
    err_linf = np.zeros(nt.size)

    for it, nti in enumerate(nt):

        sol_num = integration.rk2(tini, tend, nti, yini, fcn)
        yerr = np.absolute(cm.sol(yini, sol_num.t) - sol_num.y[0])
        
        err_l1[it]   = np.linalg.norm(yerr, 1) / (nti-1)
        err_l2[it]   = np.linalg.norm(yerr) / np.sqrt(nti-1)   
        err_linf[it] = np.linalg.norm(yerr, np.inf) 

    ref = np.array([1.e-1, 1.e-2, 1.e-3, 1.e-3, 1.e-5, 1.e-6 ])

    fig_err = figure(x_axis_type="log", y_axis_type="log", title="Error", plot_width=950, plot_height=400)
    fig_err.x(dt, err_l1, size=10, line_width=2, legend="l1 norm")
    fig_err.x(dt, err_l2, size=10, line_width=2, color="crimson", legend="l2 norm")
    fig_err.x(dt, err_linf, size=10, line_width=2, color="green", legend="linf norm")
    fig_err.line(dt, (err_l2[0]/(dt[0]*dt[0]))*dt*dt, line_width=2, legend="slope 2", color="black", line_dash='dotted')

    fig_err.legend.location = "top_left"
    fig_err.legend.click_policy="hide"

    show(fig_err)
    
show_rk2_order()